In [89]:
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
import string
string.punctuation
import re
import ast
from ast import literal_eval

In [193]:
data = pd.read_csv('/Users/zujiewu/Desktop/Zujie’s MacBook Air/ISE350/project/data/raw_data_with_hashtags.csv')

In [194]:
data

,Unnamed: 0,tweet,hashtags
0,0,"['covid19', 'era', 'normal', 'influenza', 'sea...","['covid19', 'influenza', 'greenpass', 'supergr..."
1,1,"['brussel', 'march', 'greenpass', 'novaccinepa...","['greenpass', 'novaccinepassports', 'novaccine..."
2,2,"['itali', 'tomorrow', 'go', 'creat', 'even', '...","['apartheid', 'greenpass', 'nazicovidism']"
3,3,"['histori', 'back', 'fjb', 'paperspleas', 'gre...","['fjb', 'papersplease', 'greenpass']"
4,4,"['least', 'day', 'think', 'myriad', 'italian',...","['christians', 'greenpass']"
...,...,...,...
15861,15862,"['gear', 'vaccinepassport', 'canada', 'ottawa'...","['VaccinePassport', 'Canada', 'Ottawa', 'CBC',..."
15862,15863,"['hhsgov', '😂🤪🤣', 'ador', 'don’t', 'let', 'any...","['Covid', 'coronavirus', 'Corona', 'coronaviru..."
15863,15864,"['dear', 'joebiden', 'think', 'vaccin', 'manda...","['America', 'timcast', 'VaccinePassport', 'Vac..."
15864,15865,"['im', 'bit', 'concern', 'actual', 'queen', 'c...","['Queen', 'GlasgowCop26', 'VaccinePassport', '..."


In [195]:
#defining the function to remove punctuation and emoji

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def remove_emoji(text):
    emojifree = emoji_pattern.sub(r'', text)
    return emojifree
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
data['clean_msg']= data['tweet'].apply(lambda x:remove_punctuation(x))
data['noemoji_msg']= data['clean_msg'].apply(lambda x:remove_punctuation(x))

In [196]:
#Lowering the text: 
data['msg_lower']= data['noemoji_msg'].apply(lambda x: x.lower())

In [197]:
#tokenization
def tokenization(text):
    tokens = re.split(r"\s+",text)
    return tokens
#applying function to the column
data['msg_tokenied']= data['msg_lower'].apply(lambda x: tokenization(x))

In [198]:
#stopwords removel
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
data['no_stopwords']= data['msg_tokenied'].apply(lambda x:remove_stopwords(x))

In [199]:
#stemming
porter_stemmer = PorterStemmer()
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text
data['msg_stemmed']=data['no_stopwords'].apply(lambda x: stemming(x))

In [200]:
#Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
data['msg_lemmatized']=data['msg_stemmed'].apply(lambda x:lemmatizer(x))

explode hashtags

In [203]:
df_cleaned = data[['msg_lemmatized','hashtags']]

In [204]:
df_cleaned = df_cleaned.rename(columns={"msg_lemmatized": "tweet"})

In [205]:
l = []
s = []
for index, row in df1.iterrows():
    ht = row['hashtags'].replace("\'", "\"")
    l.append(ast.literal_eval(ht))
    s.append(row['tweet'])

In [206]:
df_new = pd.DataFrame()
df_new['hashtags'] = l
df_new['tweet'] = s

In [207]:
df_new.explode('hashtags')

,hashtags,tweet
0,covid19,"[covid19, era, normal, influenza, season, lot,..."
0,influenza,"[covid19, era, normal, influenza, season, lot,..."
0,greenpass,"[covid19, era, normal, influenza, season, lot,..."
0,supergreenpass,"[covid19, era, normal, influenza, season, lot,..."
1,greenpass,"[brussel, march, greenpass, novaccinepassport,..."
...,...,...
15865,CDC,"[dr, fleme, covid, caus, solut, covid19, covid..."
15865,NaturalImmunity,"[dr, fleme, covid, caus, solut, covid19, covid..."
15865,ProtectYourFamily,"[dr, fleme, covid, caus, solut, covid19, covid..."
15865,RemdisivirKills,"[dr, fleme, covid, caus, solut, covid19, covid..."


In [208]:
df_new.to_csv('/Users/zujiewu/Desktop/Zujie’s MacBook Air/ISE350/project/data/cleaned_data.csv')